In [ ]:
import numpy as np
import os
import torch
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, Dataset, Subset
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import transforms as T
from torchvision.transforms import ToTensor 
from utils.train import train_model,set_dataset

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
target_model = torch.load("./models/victims/Cifar10_Resnet18.pt",map_location='cpu')
target_model = target_model.to(device)

In [ ]:
ct = 0
for child in target_model.children():
  ct += 1
  if ct < 3:
      for param in child.parameters():
          param.requires_grad = False

In [ ]:
#setting train hyper-parameter
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(target_model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
train_data, test_data, class_num = set_dataset("MNIST")

In [ ]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(
                 dataset=train_data,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_data,
                batch_size=batch_size,
                shuffle=True)

In [ ]:
for epochs in [20,40,60,80]:
    model = train_model(target_model, criterion, optimizer_ft, exp_lr_scheduler,device, train_loader, test_loader, epochs)
    torch.save(model,'./models/attack_model/Transfer_'+str(epochs)+'.pt')